# Exceptions
Lasciate per conoscenza personale

## Definition
When something goes wrong an exception is raised. For example, if you try to divide by zero, `ZeroDivisionError` is raised or if you try to access a nonexistent key in a dictionary, `KeyError` is raised.

In [ ]:
empty_dict = {}

# Uncomment to see the traceback
# empty_dict['key']  

## try - except
If you know that a block of code can fail in some manner, you can use `try-except` structure to handle potential exceptions in a desired way.

In [ ]:
# Let's try to open a file that does not exist
file_name = 'not_existing.txt'

try:
    with open(file_name, 'r') as my_file:
        print('File is successfully open') 
        
except FileNotFoundError as e:
    print(e)

[Errno 2] No such file or directory: 'not_existing.txt'


If you don't know the type of exceptions that a code block can possibly raise, you can use `Exception` which catches all exceptions. In addition, you can have multiple `except` statements.

In [ ]:
def calculate_division(var1, var2):
    result = 0.0
    try:
        result = var1 / var2
    except ZeroDivisionError as e:
        print(e)
    except TypeError as e:
        print(e)
    return result

print(calculate_division(3, 0))
print(calculate_division(3, '0'))

division by zero
0.0
unsupported operand type(s) for /: 'int' and 'str'
0.0


## Delegation

`try-except` can be also in outer scope:

In [ ]:
def calculate_division(var1, var2):
    return var1 / var2

try:
    calculate_division(3, 0)
except ZeroDivisionError as e:
    print(e)
except TypeError as e:
    print(e)

division by zero


In [ ]:
def calculate_division(var1, var2):
    return var1 / var2

def process(var1, var2):
    # other computations
    return calculate_division(var1, var2) 

try:
    process(3, 0)
except ZeroDivisionError as e:
    print(e)
except TypeError as e:
    print(e)

division by zero


## Raising exceptions

We can use the *raise* keyword to throw an exception if a condition occurs. The statement can be complemented with a custom exception. Using standard exceptions is nevertheless preferred. Refer to [https://docs.python.org/3/library/exceptions.html](https://docs.python.org/3/library/exceptions.html) for the full taxonomy of exceptions.

In [ ]:
def calculate_division(var1, var2):
    result = 0.0
    
    try:
        result = var1 / var2
    except ZeroDivisionError:
        raise ValueError('Zero-division error')
    except TypeError:
        raise ValueError('Type error')
    return result

try:
    calculate_division(2, '3')
except ValueError as e:
    print(e)

Type error


## try - except - else

The optional *else* clause is executed if and when control flows off the end of the try clause.
Control *flows off the end* except in the case of an exception or the execution of a return, continue, or break statement.

In [ ]:
def calculate_division(var1, var2):
    return var1 / var2

In [ ]:
# Don't do this!
exception_occured = False
try:
    calculate_division(1, 0)
except ZeroDivisionError:
    exception_occured = True
except TypeError:
    exception_occured = True
    
if not exception_occured:
    print('All went well!')
else:
    print('Something happened!')

Something happened!


In [ ]:
# Do this!
try:
    calculate_division(1, 0)
except ZeroDivisionError:
    print('Something happened!')
except TypeError:
    print('Something happened!')
else:
    print('All went well!')

Something happened!


## try - except - finally
For scenarios where you want to do something always, even when there are exceptions. A *finally* clause is always executed before leaving the try statement, whether an exception has occurred or not. When an exception has occurred in the try clause and has not been handled by an except clause (or it has occurred in a except or else clause), it is re-raised after the finally clause has been executed. The finally clause is also executed *on the way out* when any other clause of the try statement is left via a break, continue or return statement. 

You can also have `try`-`except`-`else`-`finally` structure. In cases where exception is not raised inside `try`, `else` will be executed before `finally`. If there is an expection, `else` block is not executed.

In [ ]:
def calculate_division(var1, var2):
    return var1 / var2

try:
    calculate_division(1, 0)
except ZeroDivisionError:
    print('Something happened!')
except TypeError:
    print('Something happened!')
else:
    print('All went well!')
finally:
    print('Always do it!')

Something happened!
Always do it!


## Summarizing
![alt](images/exceptions.png)

# File I/O

## Paths

I Path sono un'astrazione dei percorsi ai file che sono presenti in ogni dispositivo.  
I percorsi infatti sono dipendenti dal sistema operativo, ad esempio i percorsi in Windows iniziano con l'identificatore di un disco e le varie cartelle sono separate da backslash ('\'), mentre su sistemi Unix-based come Linux o MacOS non c'è un'identificatore di disco iniziale e le cartelle sono separate da forward slash '/'.  
  
Ci sono anche delle differenze di formato tra file e file: su Windows il terminatore di riga è mappato con la sequenza di caratteri ASCII "CR LF", mentre su Linux si usa il solo "CR" mentre su MacOS si usa il solo "LF"

In [1]:
import os

current_file = os.path.realpath('01 - Fondamenti di Python.ipynb')  
print('file: {}'.format(current_file))

current_dir = os.path.dirname(current_file)  
print('directory: {}'.format(current_dir))

data_dir = os.path.join(current_dir, 'resources')
print('data: {}'.format(data_dir))

file: /home/matteo/MEGA/POC Fanti/learn-python-core/slides/01 - Fondamenti di Python.ipynb
directory: /home/matteo/MEGA/POC Fanti/learn-python-core/slides
data: /home/matteo/MEGA/POC Fanti/learn-python-core/slides/resources


## Controlla il Path

In [ ]:
print('exists: {}'.format(os.path.exists(current_dir)))
print('is file: {}'.format(os.path.isfile(current_dir)))
print('is directory: {}'.format(os.path.isdir(current_dir)))

exists: True
is file: False
is directory: True


## Lettura di file

L'istruzione [`with`](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) serve per ottenere un [gestore di contesto (context manager)](https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers) che verrà utilizzato come contesto di esecuzione per i comandi all'interno di `with`. I gestori di contesto garantiscono che determinate operazioni vengano eseguite all'uscita dal contesto. 

In questo caso, il gestore di contesto garantisce che `file_path.close()` venga chiamato implicitamente all'uscita dal contesto. Questo semplifica la vita agli sviluppatori: non è necessario ricordarsi di chiudere esplicitamente il file aperto né preoccuparsi che si verifichi un'eccezione mentre il file è aperto. I file non chiusi possono essere fonte di perdite di risorse. Pertanto, è preferibile utilizzare la struttura `with open()` quando si lavora con l'I/O.

In [ ]:
# Don't do this!
file_path = os.path.join(data_dir, 'cars.txt')
simple_file = open(file_path, 'r')

for line in simple_file:
    print(line.strip())
simple_file.close()  # This has to be called explicitly 

BMW, M3, 120
Toyota, Supra, 130
Nissan, GTR, 140


In [ ]:
# Do this!
file_path = os.path.join(data_dir, 'cars.txt')

with open(file_path, 'r') as simple_file:
    for line in simple_file:
        print(line.strip())

BMW, M3, 120
Toyota, Supra, 130
Nissan, GTR, 140


## Scrittura di file

In [ ]:
new_file_path = os.path.join(data_dir, 'new_file.txt')

with open(new_file_path, 'w') as my_file:
    my_file.write('This is my first file that I wrote with Python.')

Controlla che sia stato creato il file `new_file.txt` con il contenuto specificato. Poi esegui la cella seguente per cancellarne il contenuto.

In [ ]:
if os.path.exists(new_file_path):  # make sure it's there
    os.remove(new_file_path)